In [2]:
import kmapper as km 
from kmapper.jupyter import display
import umap.umap_ as umap
import sklearn
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd
from sklearn.cluster import KMeans
import seaborn as sns
from geopy.geocoders import Nominatim
import ssl
import certifi

In [3]:
dfinicial = pd.read_csv('COVID19MEXICO.csv')
dfinicial.dtypes

FECHA_ACTUALIZACION      object
ID_REGISTRO              object
ORIGEN                    int64
SECTOR                    int64
ENTIDAD_UM                int64
SEXO                      int64
ENTIDAD_NAC               int64
ENTIDAD_RES               int64
MUNICIPIO_RES             int64
TIPO_PACIENTE             int64
FECHA_INGRESO            object
FECHA_SINTOMAS           object
FECHA_DEF                object
INTUBADO                  int64
NEUMONIA                  int64
EDAD                      int64
NACIONALIDAD              int64
EMBARAZO                  int64
HABLA_LENGUA_INDIG        int64
INDIGENA                  int64
DIABETES                  int64
EPOC                      int64
ASMA                      int64
INMUSUPR                  int64
HIPERTENSION              int64
OTRA_COM                  int64
CARDIOVASCULAR            int64
OBESIDAD                  int64
RENAL_CRONICA             int64
TABAQUISMO                int64
OTRO_CASO                 int64
TOMA_MUE

Falta agregar: Estado (Nacimiento) Latitud y Longitud, Número de Casos Confirmados (hasta la fecha, por estado), Estado de Residencia Latitud y Longitud, Municipio de Residencia Latitud y Longitud

In [4]:
df = dfinicial[["FECHA_ACTUALIZACION","ENTIDAD_NAC", "ENTIDAD_RES","MUNICIPIO_RES"]]

Borramos todos los valores nulos, estos aparecen en la base de datos como 99 o 999 según el diccionario, por lo que tenemos que tratarlos como un valor diferente de nulo.

In [5]:
df['bools'] = df["ENTIDAD_NAC"]>95
df = df.drop(df[df["bools"]==True].index)
df['bool1s'] = df["ENTIDAD_RES"]>95
df = df.drop(df[df["bool1s"]==True].index)
df['bool2s'] = df["MUNICIPIO_RES"]>996
df = df.drop(df[df["bool2s"]==True].index)
df1 = df[["FECHA_ACTUALIZACION","ENTIDAD_NAC", "ENTIDAD_RES","MUNICIPIO_RES"]]
df1[["ENTIDAD_RES","MUNICIPIO_RES"]]=df1[["ENTIDAD_RES","MUNICIPIO_RES"]].astype(str)
df1["ID_RES"] = df1["ENTIDAD_RES"] + df1["MUNICIPIO_RES"]
df2 = df1[["FECHA_ACTUALIZACION", "ENTIDAD_NAC", "ID_RES"]]
print(df2.head())

/var/folders/0n/92mz65hj527d4cmt71d5s0q00000gn/T/ipykernel_50435/1297708964.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['bools'] = df["ENTIDAD_NAC"]>95


  FECHA_ACTUALIZACION  ENTIDAD_NAC ID_RES
0          2024-04-16           15     33
1          2024-04-16           25   1817
2          2024-04-16           14  14120
3          2024-04-16            3     33
4          2024-04-16            7   7107


/var/folders/0n/92mz65hj527d4cmt71d5s0q00000gn/T/ipykernel_50435/1297708964.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[["ENTIDAD_RES","MUNICIPIO_RES"]]=df1[["ENTIDAD_RES","MUNICIPIO_RES"]].astype(str)
/var/folders/0n/92mz65hj527d4cmt71d5s0q00000gn/T/ipykernel_50435/1297708964.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["ID_RES"] = df1["ENTIDAD_RES"] + df1["MUNICIPIO_RES"]


In [6]:
Dic_MN = pd.read_excel("201128 Catalogos.xlsx", sheet_name='Catálogo MUNICIPIOS')
Dic_MN['bools']= Dic_MN["CLAVE_MUNICIPIO"]>995
Dic_MN = Dic_MN.drop(Dic_MN[Dic_MN["bools"]==True].index)
Dic_MN['bool1s']= Dic_MN["CLAVE_ENTIDAD"]>995
Dic_MN = Dic_MN.drop(Dic_MN[Dic_MN["bool1s"]==True].index)
Dic_MN[["CLAVE_ENTIDAD","CLAVE_MUNICIPIO"]]=Dic_MN[["CLAVE_ENTIDAD","CLAVE_MUNICIPIO"]].astype(str)
Dic_MN["ID"] = Dic_MN["CLAVE_ENTIDAD"] + Dic_MN["CLAVE_MUNICIPIO"]
Dic_MN1 = Dic_MN[["MUNICIPIO","CLAVE_ENTIDAD","ID"]]
(Dic_MN1.value_counts()==2).unique() #Para ver si no hay repetidos

array([False])

In [7]:
Dic_MN1.head()

,MUNICIPIO,CLAVE_ENTIDAD,ID
0,AGUASCALIENTES,1,11
1,ASIENTOS,1,12
2,CALVILLO,1,13
3,COSÍO,1,14
4,JESÚS MARÍA,1,15


In [8]:
Dic_EN = pd.read_excel("201128 Catalogos.xlsx", sheet_name='Catálogo de ENTIDADES')
Dic_EN['bools']= Dic_EN["CLAVE_ENTIDAD"]>95
Dic_EN = Dic_EN.drop(Dic_EN[Dic_EN["bools"]==True].index)
Dic_EN1 = Dic_EN[["ENTIDAD_FEDERATIVA","CLAVE_ENTIDAD"]]
(Dic_EN1.value_counts()==2).unique() #Para ver si no hay repetidos
Dic_EN1.head()

,ENTIDAD_FEDERATIVA,CLAVE_ENTIDAD
0,AGUASCALIENTES,1
1,BAJA CALIFORNIA,2
2,BAJA CALIFORNIA SUR,3
3,CAMPECHE,4
4,COAHUILA DE ZARAGOZA,5


In [74]:
df2.head()

,FECHA_ACTUALIZACION,ENTIDAD_NAC,ID_RES
0,2024-04-16,15,33
1,2024-04-16,25,1817
2,2024-04-16,14,14120
3,2024-04-16,3,33
4,2024-04-16,7,7107


In [10]:
def obtener_lat_long(nombre_municipio,nombre_estado):
    # Configurar la sesión de solicitudes con verificación SSL personalizada
    ctx = ssl.create_default_context(cafile=certifi.where())
    geolocalizador = Nominatim(user_agent="aplicacion_de_ejemplo", ssl_context=ctx)
    ubicacion = geolocalizador.geocode(f"{nombre_municipio},{nombre_estado}, Mexico")
    if ubicacion:
        return ubicacion.latitude, ubicacion.longitude
    else:
        return None, None

In [79]:
def obtener_estado(clave_entidad):
    estado = Dic_EN1[Dic_EN1['CLAVE_ENTIDAD'] == clave_entidad]
    if not estado.empty:
        return estado.iloc[0]  # Devuelve el primer resultado encontrado
    else:
        return None  # Devuelve None si no se encuentra ningún estado con esa clave

# Función para obtener el municipio por ID
def obtener_municipio(id_municipio):
    municipio = Dic_MN1[Dic_MN1['ID'] == str(id_municipio)]
    if not municipio.empty:
        return municipio.iloc[0]  # Devuelve el primer resultado encontrado
    else:
        return None  # Devuelve None si no se encuentra ningún municipio con ese ID


In [17]:
Dic_MN1.reset_index(drop=True, inplace=True)
Dic_MN1.head(15)

,MUNICIPIO,CLAVE_ENTIDAD,ID
0,AGUASCALIENTES,1,11
1,ASIENTOS,1,12
2,CALVILLO,1,13
3,COSÍO,1,14
4,JESÚS MARÍA,1,15
5,PABELLÓN DE ARTEAGA,1,16
6,RINCÓN DE ROMOS,1,17
7,SAN JOSÉ DE GRACIA,1,18
8,TEPEZALÁ,1,19
9,EL LLANO,1,110


In [36]:
obtener_estado(int(Dic_MN1['CLAVE_ENTIDAD'][2]))

ENTIDAD_FEDERATIVA    AGUASCALIENTES
CLAVE_ENTIDAD                      1
Name: 0, dtype: object

In [40]:
Municipio_ID = []
Entidad_ID = []

for i in range(len(Dic_MN1)):
    MunID = obtener_municipio(Dic_MN1['ID'][i])
    Municipio_ID.append(MunID)
    EntID = obtener_estado(int(Dic_MN1['CLAVE_ENTIDAD'][i]))
    Entidad_ID.append(EntID)

MunicipiosDF = pd.concat(Municipio_ID)
EntidadDF = pd.concat(Entidad_ID)

EMDF = MunicipiosDF[['MUNICIPIO']],EntidadDF[['ENTIDAD_FEDERATIVA']]

In [54]:
latitudes = []
longitudes = []
for municipio, entidad in zip(EMDF[0], EMDF[1]):
    latitud, longitud = obtener_lat_long(municipio, entidad)
    latitudes.append(latitud)
    longitudes.append(longitud)

Dic_MN1["Latitud"] = latitudes
Dic_MN1["Longitud"] = longitudes



/var/folders/0n/92mz65hj527d4cmt71d5s0q00000gn/T/ipykernel_50435/2736030652.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Dic_MN1["Latitud"] = latitudes
/var/folders/0n/92mz65hj527d4cmt71d5s0q00000gn/T/ipykernel_50435/2736030652.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Dic_MN1["Longitud"] = longitudes


In [75]:
Dic_MN1.head()
MunicipiosLL = Dic_MN1[["ID","MUNICIPIO","Latitud","Longitud"]]
Ent = pd.Series(list(EntidadDF['ENTIDAD_FEDERATIVA']))
MunicipiosLL1=pd.concat([MunicipiosLL,Ent], axis=1)
MunicipiosLL1 = MunicipiosLL1.rename(columns={0: 'ENTIDAD'})
MunicipiosLL1.head(15)

,ID,MUNICIPIO,Latitud,Longitud,ENTIDAD
0,11,AGUASCALIENTES,21.880487,-102.296719,AGUASCALIENTES
1,12,ASIENTOS,22.132107,-102.095256,AGUASCALIENTES
2,13,CALVILLO,21.846512,-102.718573,AGUASCALIENTES
3,14,COSÍO,22.370515,-102.312009,AGUASCALIENTES
4,15,JESÚS MARÍA,21.934556,-102.468452,AGUASCALIENTES
5,16,PABELLÓN DE ARTEAGA,22.104934,-102.301624,AGUASCALIENTES
6,17,RINCÓN DE ROMOS,22.229364,-102.322985,AGUASCALIENTES
7,18,SAN JOSÉ DE GRACIA,22.142627,-102.530136,AGUASCALIENTES
8,19,TEPEZALÁ,22.243516,-102.192721,AGUASCALIENTES
9,110,EL LLANO,21.928670,-102.000728,AGUASCALIENTES
